# Automated Factor Process System 

need to define:

In [1]:
# define symbol to collect data
symbol, interval, delay_minutes, start_year, start_month, start_day, end_year, end_month, end_day = 'BTCUSDT', '1h', 1, 2021, 1, 1, 2024, 6, 1
### (end not included) #such as: 'BTCUSDT', '1h', 1, 2021, 1, 1, 2024, 6, 1

# calculate return

# feature processing method
method = 'standardize' 
### method = normalize/standardize

# train_test_split
train_start, train_end, test_start, test_end = '2021-01-01', '2024-01-01', '2024-01-01', '2024-06-01' # can change

# simply strategy threshold and cost
threshold = 0.002
cost = 0.002

## 1. Data Collection

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import a_binance_data_collection as binance_data_collection
import b1_calculate_return as cr
import b2_class_calculate_return as ccr
import c1_calculate_specific_factor as csf # we will change feature engineering(factor)
import d_feature_preprocssing as feapre
import e_train_test_split as tts 
import model_construction_1_linear as model_construction_1_linear # we will change model
import f1_basic_strategy_backtest as bsb # we will change stategy
import f2_basic_strategy_backtesting_class as bsbc

In [3]:
binance_data = binance_data_collection.binance_data_collection(symbol, interval, delay_minutes, start_year, start_month, start_day, end_year, end_month, end_day) # symbol, interval, delay_minutes, start_year, start_month, start_day, end_year, end_month, end_day(end not included)

Data has been saved to BTCUSDT_2021_1_1_to_2024_6_1_with_1h.csv


In [4]:
data  = pd.read_csv(binance_data)
data['date_time'] = pd.to_datetime(data['date_time'])
data['real_time'] = pd.to_datetime(data['real_time'])
print(data)

# simply process
for column in data.columns:
    median_value = data[column].median()
    data[column] = data[column].fillna(median_value)
print(data.isna().sum())

                date_time           real_time      timestamp      open  \
0     2021-01-01 00:01:00 2021-01-01 00:00:00  1609459200000  28948.19   
1     2021-01-01 01:01:00 2021-01-01 01:00:00  1609462800000  29015.00   
2     2021-01-01 02:01:00 2021-01-01 02:00:00  1609466400000  29443.65   
3     2021-01-01 03:01:00 2021-01-01 03:00:00  1609470000000  29236.58   
4     2021-01-01 04:01:00 2021-01-01 04:00:00  1609473600000  29302.11   
...                   ...                 ...            ...       ...   
29923 2024-05-31 19:01:00 2024-05-31 19:00:00  1717182000000  67456.20   
29924 2024-05-31 20:01:00 2024-05-31 20:00:00  1717185600000  67720.00   
29925 2024-05-31 21:01:00 2024-05-31 21:00:00  1717189200000  67755.00   
29926 2024-05-31 22:01:00 2024-05-31 22:00:00  1717192800000  67602.50   
29927 2024-05-31 23:01:00 2024-05-31 23:00:00  1717196400000  67537.20   

           high       low     close     volume  quote_asset_volume  \
0      29055.00  28706.00  29015.00   803

In [5]:
# # plot data
# def plot_data(data, x_column, y_column, title):
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=data[x_column], y=data[y_column], mode='lines', name=y_column))
#     fig.update_layout(title=title, xaxis_title=x_column, yaxis_title=y_column)
#     fig.show()

# for column in data.columns[3:14]:
#     plot_data(data, 'date_time', column, f'{column}')

## 2. Factor calcualtion 

In [6]:
# # calcualte return
period_labels = [interval]
data = cr.compute_returns(data, period_labels=period_labels)

# period_labels = [interval]
# data = ccr.compute_returns(data, period_labels=period_labels, cost = cost)
data

,real_time,timestamp,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,return_1h
date_time,,,,,,,,,,,,
2021-01-01 00:01:00,2021-01-01 00:00:00,1609459200000,28948.19,29055.00,28706.00,29015.00,8037.588,2.321646e+08,65942.0,3983.842,1.151312e+08,NaN
2021-01-01 01:01:00,2021-01-01 01:00:00,1609462800000,29015.00,29546.42,28975.46,29448.40,19543.401,5.736394e+08,127126.0,10386.352,3.048663e+08,0.014937
2021-01-01 02:01:00,2021-01-01 02:00:00,1609466400000,29443.65,29500.00,29159.16,29237.06,10049.720,2.946641e+08,69148.0,4513.355,1.323593e+08,-0.007177
2021-01-01 03:01:00,2021-01-01 03:00:00,1609470000000,29236.58,29379.41,29174.95,29302.11,5579.452,1.633982e+08,47082.0,2969.050,8.697145e+07,0.002225
2021-01-01 04:01:00,2021-01-01 04:00:00,1609473600000,29302.11,29422.32,29086.00,29237.07,8351.683,2.443911e+08,59332.0,3661.258,1.071895e+08,-0.002220
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31 19:01:00,2024-05-31 19:00:00,1717182000000,67456.20,68059.40,67388.20,67719.90,12522.881,8.486403e+08,145735.0,6950.388,4.710058e+08,0.003911
2024-05-31 20:01:00,2024-05-31 20:00:00,1717185600000,67720.00,67834.40,67633.10,67755.00,3592.345,2.433091e+08,57772.0,1701.252,1.152285e+08,0.000518
2024-05-31 21:01:00,2024-05-31 21:00:00,1717189200000,67755.00,67763.00,67557.70,67602.30,2543.573,1.720847e+08,42362.0,1088.662,7.364917e+07,-0.002254


In [ ]:

# calculate specific factor
df = data.copy()
period_labels = period_labels

start_idx = df.columns.get_loc('volume') + 1
return_col = df.filter(like='return').columns.tolist()
end_idx = min(df.columns.get_loc(col) for col in return_col)
specific_factor_col = df.columns[start_idx:end_idx]

df_fac = csf.calculate_specific_factor(df, specific_factor_col, period_labels = period_labels)
# print(df_fac)


In [ ]:
# from tsfresh import extract_features
# from tsfresh.utilities.dataframe_functions import impute
# from tsfresh.utilities.distribution import MultiprocessingDistributor
# from tsfresh import extract_features
# from tsfresh.utilities.dataframe_functions import roll_time_series
# train_start, train_end, test_start, test_end = train_start, train_end, test_start, test_end
# train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test = tts.train_test_split(df_fac, train_start, train_end, test_start, test_end)
# timeseries = train_df[['close', 'volume', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']].copy()
# timeseries['code'] = 1
# timeseries.reset_index(inplace=True)

# data_roll = roll_time_series(timeseries, column_id='code', column_sort='date_time', max_timeshift=3, min_timeshift=1).drop(columns=['code'])


# distributor = MultiprocessingDistributor(n_workers=1, disable_progressbar=False, progressbar_title="Feature Extraction")
    
# extracted_features = extract_features(data_roll, column_id="id", column_sort="date_time", chunksize=60,distributor=distributor)
# impute(extracted_features)

# print(extracted_features)
# extracted_features.to_csv('extracted_features.csv')

In [ ]:

# calculate commom factor
## use tsfresh

## use featuretools
### soon....

## 3.Feature preprocessing 

In [ ]:
feature_df = df_fac.copy()

# return_data & feature_data processing
return_col = feature_df.filter(like='return').columns.tolist()
feature_df = feapre.handle_return(feature_df, return_col)
max_return_index = max(feature_df.columns.get_loc(col) for col in return_col)
feature_col = feature_df.columns[max_return_index + 1:].tolist()
feature_df = feapre.feature_preprocessing(feature_df, feature_col, method = method) # method = normalize/standardize

print(feature_df)

## 4.Model construction 

In [ ]:
model_df = feature_df.copy()
train_start, train_end, test_start, test_end = train_start, train_end, test_start, test_end
train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test = tts.train_test_split(model_df, train_start, train_end, test_start, test_end)


In [ ]:
# import model_construction_1_linear as model_construction_1_linear
# result_df, test_df = model_construction_1_linear.model_construction_linear(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [ ]:
# import model_construction_2_lasso as model_construction_2_lasso
# result_df, test_df = model_construction_2_lasso.model_construction_lasso(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)


In [ ]:
# import model_construction_3_ridge as model_construction_3_ridge
# result_df, test_df = model_construction_3_ridge.model_construction_ridge(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [ ]:
# import model_construction_4_logistic as model_construction_4_logistic
# result_df, test_df = model_construction_4_logistic.model_construction_logistic(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [ ]:
# import model_construction_5_lda as model_construction_5_lda
# result_df, test_df = model_construction_5_lda.model_construction_lda(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [ ]:
# import model_construction_6_qda as model_construction_6_qda
# result_df, test_df = model_construction_6_qda.model_construction_qda(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [ ]:
import model_construction_7_decision_trees_reg as model_construction_7_decision_trees_reg
result_df, test_df = model_construction_7_decision_trees_reg.model_construction_decision_trees(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [ ]:
# import model_construction_8_decision_trees_class as model_construction_8_decision_trees_class
# result_df, test_df = model_construction_8_decision_trees_class(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [ ]:
# import model_construction_9_random_forest_reg as model_construction_9_random_forest_reg
# result_df, test_df = model_construction_9_random_forest_reg.model_construction_random_forest(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [ ]:
# import model_construction_10_random_forest_class as model_construction_10_random_forest_class
# result_df, test_df = model_construction_10_random_forest_class.model_construction_random_forest(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [ ]:
# import model_construction_11_xgboost_reg as model_construction_11_xgboost_reg
# result_df, test_df = model_construction_11_xgboost_reg.model_construction_xgboost(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [ ]:
# import model_construction_12_xgboost_class as model_construction_12_xgboost_class
# result_df, test_df = model_construction_12_xgboost_class.model_construction_xgboost(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [ ]:
# import model_construction_13_adaboost_reg as model_construction_13_adaboost_reg
# result_df, test_df = model_construction_13_adaboost_reg.model_construction_adaboost(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [ ]:
# import model_construction_14_adaboost_class as model_construction_14_adaboost_class
# result_df, test_df = model_construction_14_adaboost_class.model_construction_adaboost(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [ ]:
# import model_construction_15_gbm_reg as model_construction_15_gbm_reg
# result_df, test_df = model_construction_15_gbm_reg.model_construction_gbm(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [ ]:
# import model_construction_16_gbm_class as model_construction_16_gbm_class
# result_df, test_df = model_construction_16_gbm_class.model_construction_gbm(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [ ]:
result_df

In [ ]:
columns_to_keep = ['close']
for y_unique in result_df['Y'].unique():
    columns_to_keep.append(y_unique)
for x in result_df['X'].unique():
    for y in result_df[result_df['X'] == x]['Y'].unique():
        pred_col_name = f"{x}___{y}_pred_rtn"
        if pred_col_name not in columns_to_keep:
            columns_to_keep.append(pred_col_name)
        
model_df = test_df[columns_to_keep]
model_df

## 5.Backtesting

In [ ]:
back_df = model_df.copy()
back_df = bsb.basic_strategy_backtesting(back_df, threshold, cost)
# back_df = bsbc.basic_strategy_backtesting(back_df, cost)
back_df